<a href="https://colab.research.google.com/github/ko-bbie/bert_livedoor_classifier/blob/master/notebook/livedoor_albert_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==2.11

     |████████████████████████████████| 675kB 8.5MB/s 
     |████████████████████████████████| 890kB 23.6MB/s 
     |████████████████████████████████| 1.1MB 58.3MB/s 
     |████████████████████████████████| 3.8MB 60.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=8a57c2c47782768fbf72a483fe92dcabc85cf82f569e7a417b438e4442a65dea
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from google.colab import drive

# Google Driveをマウントする仮想マシン上のディレクトリ
DIR_DRIVE = './gdrive/'

# Google Drive上でのNotebook等の各種ファイルのパス
DIR_COLAB = DIR_DRIVE + 'My Drive/Colab Notebooks/'
DIR_PROJCET = DIR_COLAB + 'livedoor_classification/'

# Google Driveをマウント
drive.mount(DIR_DRIVE)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at ./gdrive/


In [ ]:
DATA_DIR = DIR_DRIVE + 'My Drive/nlp/data/'

## import

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import transformers
import re
import unicodedata
from tqdm import tqdm
from sklearn import model_selection
from sklearn import metrics
from transformers import AdamW, get_linear_schedule_with_warmup

## Config

In [ ]:
MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 16
EPOCHS = 10
ALBERT_PATH = "ALINEAR/albert-japanese-v2"
MODEL_PATH = DATA_DIR + "albert-model.bin"
TRAINING_FILE = DATA_DIR + "train.csv"
TOKENIZER = transformers.AutoTokenizer.from_pretrained(ALBERT_PATH)


## Utils

In [ ]:
def category_dict():
    dic = {
        "sports-watch": [1, 0, 0, 0, 0, 0, 0, 0, 0],
        "smax": [0, 1, 0, 0, 0, 0, 0, 0, 0],
        "dokujo-tsushin": [0, 0, 1, 0, 0, 0, 0, 0, 0],
        "movie-enter": [0, 0, 0, 1, 0, 0, 0, 0, 0],
        "it-life-hack": [0, 0, 0, 0, 1, 0, 0, 0, 0],
        "kaden-channel": [0, 0, 0, 0, 0, 1, 0, 0, 0],
        "peachy": [0, 0, 0, 0, 0, 0, 1, 0, 0],
        "topic-news": [0, 0, 0, 0, 0, 0, 0, 1, 0],
        "livedoor-homme": [0, 0, 0, 0, 0, 0, 0, 0, 1],
    }
    return dic


def normalize(text):
    text = normalize_unicode(text)
    text = capitalize_symbols(text)
    return text


def normalize_unicode(text, form="NFKC"):
    normalized_text = unicodedata.normalize(form, text)
    normalized_text = re.sub(r"\s", " ", normalized_text)
    return normalized_text


def capitalize_symbols(text):
    # this function is often used for mecab morph analysis
    text = text.replace("・", "")
    table = str.maketrans(dict(zip("!?(),.:;/@%&[]", "！？（）、。：；／＠％＆［］")))
    capitalized_text = text.translate(table)
    return capitalized_text

## Dataset

In [ ]:
class LivedoorDataset:
    def __init__(self, article, targets):
        self.article = article
        self.targets = targets
        self.tokenizer = TOKENIZER
        self.max_len = MAX_LEN
        self.category_dic = category_dict()

    def __len__(self):
        return len(self.article)

    def __getitem__(self, item):
        article = normalize(self.article[item])

        inputs = self.tokenizer.encode_plus(
            article, add_special_tokens=True
        )

        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]
        token_type_ids = inputs["token_type_ids"]

        padding_len = self.max_len - len(ids)
        if padding_len >= 0:
            ids = ids + [0] * padding_len
            mask = mask + [0] * padding_len
            token_type_ids = token_type_ids + [0] * padding_len
        else:
            half_len = int(self.max_len / 2)
            ids = ids[:half_len] + ids[-half_len:]
            mask = mask[:half_len] + mask[-half_len:]
            token_type_ids = token_type_ids[:half_len] + token_type_ids[-half_len:]

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "targets": torch.tensor(
                self.category_dic.get(self.targets[item]), dtype=torch.long
            ),
            "orig_targets": self.targets[item],
        }

## Model

In [ ]:
class AlbertBaseJapanese(nn.Module):
    def __init__(self):
        super(AlbertBaseJapanese, self).__init__()
        self.albert = transformers.AlbertModel.from_pretrained(ALBERT_PATH)
        self.drop = nn.Dropout(0.3)
        self.out = nn.Linear(768, 9)

    def forward(self, ids, mask, token_type_ids):
        _, pooled_output = self.albert(
            ids, attention_mask=mask, token_type_ids=token_type_ids
        )
        bo = self.drop(pooled_output)
        return self.out(bo)

## Engine

In [ ]:
def loss_fn(outputs, targets):
    return nn.BCEWithLogitsLoss()(outputs, targets)


def train_fn(data_loader, model, optimizer, device, scheduler):
    model.train()
    tk0 = tqdm(data_loader, total=len(data_loader))
    for bi, d in enumerate(tk0):
        ids = d["ids"]
        mask = d["mask"]
        token_type_ids = d["token_type_ids"]
        targets = d["targets"]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)

        loss = loss_fn(torch.softmax(outputs, dim=1), targets)
        loss.backward()
        optimizer.step()
        scheduler.step()


def eval_fn(data_loader, model, device):
    model.eval()
    fin_targets = []
    fin_outputs = []

    for bi, d in enumerate(data_loader):
        ids = d["ids"]
        mask = d["mask"]
        token_type_ids = d["token_type_ids"]
        targets = d["targets"]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.long)

        outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
        fin_targets.extend(targets.argmax(1).cpu().detach().numpy().tolist())
        fin_outputs.extend(outputs.argmax(1).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

## Train

In [ ]:
def run():
    df = pd.read_csv(TRAINING_FILE).fillna("none")

    df_train, df_valid = model_selection.train_test_split(
        df, test_size=0.1, random_state=42, stratify=df.category.values
    )

    df_train = df_train.reset_index(drop=True)
    df_valid = df_valid.reset_index(drop=True)

    train_dataset = LivedoorDataset(
        article=df_train.article.values, targets=df_train.category.values
    )

    train_data_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=TRAIN_BATCH_SIZE, num_workers=4
    )

    valid_dataset = LivedoorDataset(
        article=df_valid.article.values, targets=df_valid.category.values
    )

    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=VALID_BATCH_SIZE, num_workers=1
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = AlbertBaseJapanese()
    model.to(device)

    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {
            "params": [
                p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.001,
        },
        {
            "params": [
                p for n, p in param_optimizer if any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.0,
        },
    ]

    num_train_steps = int(len(df_train) / TRAIN_BATCH_SIZE * EPOCHS)
    optimizer = AdamW(optimizer_parameters, lr=3e-5)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=0, num_training_steps=num_train_steps
    )

    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
        model.to(device)


    best_accuracy = 0
    for epoch in range(EPOCHS):
        train_fn(train_data_loader, model, optimizer, device, scheduler)
        outputs, targets = eval_fn(valid_data_loader, model, device)
        accuracy = metrics.accuracy_score(targets, outputs)
        print(f"Accuracy Score = {accuracy}")
        if accuracy > best_accuracy:
            torch.save(model.state_dict(), MODEL_PATH)
            best_accuracy = accuracy
        print(f"epoch = {epoch}, best_accuracy = {best_accuracy}")

In [ ]:
run()


100%|██████████| 166/166 [04:10<00:00,  1.51s/it]

  0%|          | 0/166 [00:00<?, ?it/s]

Accuracy Score = 0.8728813559322034
epoch = 0, best_accuracy = 0.8728813559322034



100%|██████████| 166/166 [04:24<00:00,  1.60s/it]

  0%|          | 0/166 [00:00<?, ?it/s]

Accuracy Score = 0.9288135593220339
epoch = 1, best_accuracy = 0.9288135593220339



100%|██████████| 166/166 [04:25<00:00,  1.60s/it]

  0%|          | 0/166 [00:00<?, ?it/s]

...

Accuracy Score = 0.9694915254237289
epoch = 7, best_accuracy = 0.9694915254237289



100%|██████████| 166/166 [04:24<00:00,  1.59s/it]

  0%|          | 0/166 [00:00<?, ?it/s]

Accuracy Score = 0.9728813559322034
epoch = 8, best_accuracy = 0.9728813559322034



100%|██████████| 166/166 [04:24<00:00,  1.59s/it]


Accuracy Score = 0.9694915254237289
epoch = 9, best_accuracy = 0.9728813559322034
